In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
from pyproj import CRS



In [4]:
# remove lakes that are completely contained by new lake geometries in the GLAKES dataset

hybas_id_list = [
    1020000010, 1020011530, 1020018110, 1020021940, 1020027430, 1020034170, 1020035180, 1020040190,
    2020000010, 2020003440, 2020018240, 2020024230, 2020033490, 2020041390, 2020057170, 2020065840, 2020071190,
    3020000010, 3020003790, 3020005240, 3020008670, 3020009320, 3020024310,
    4020000010, 4020006940, 4020015090, 4020024190, 4020034510, 4020050210, 4020050220, 4020050290, 4020050470,
    5020000010, 5020015660, 5020037270, 5020049720, 5020082270, 
    6020000010, 6020006540, 6020008320, 6020014330, 6020017370, 6020021870, 6020029280,
    7020000010, 7020014250, 7020021430, 7020024600, 7020038340, 7020046750, 7020047840, 7020065090,
    8020000010, 8020008900, 8020010700, 8020020760, 8020022890, 8020032840, 8020044560,
    9020000010
]
hylak_folder = '/WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin'
hylak_filename_pattern = 'hylak_unbuffered_updated_{hybas_id}.shp'
hylak_no_contained_save_folder = '/WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new'
if not os.path.exists(hylak_no_contained_save_folder):
    os.makedirs(hylak_no_contained_save_folder)
hylak_no_contained_save_filename_pattern = 'hylak_unbuffered_updated_no_contained_{hybas_id}.shp'

from pandarallel import pandarallel
print('Initializing pandarallel...')
pandarallel.initialize(progress_bar=False, nb_workers=12, use_memory_fs = False)

for hybas_id in hybas_id_list:
    print(f'Processing {hybas_id}')
    current_gdf = gpd.read_file(os.path.join(hylak_folder, hylak_filename_pattern.format(hybas_id=hybas_id)))
    # Create spatial index for the GeoDataFrame
    sindex = current_gdf.sindex
    geometry_col_name = current_gdf.geometry.name
    # List to hold indices of geometries to keep
    keep_indices = []

    # Load the GeoDataFrame
    current_gdf = gpd.read_file(os.path.join(hylak_folder, hylak_filename_pattern.format(hybas_id=hybas_id)))

    # Create spatial index for the GeoDataFrame
    sindex = current_gdf.sindex
    geometry_col_name = current_gdf.geometry.name

    def check_containment(idx, row):
        current_centroid = row[geometry_col_name].centroid
        possible_matches_index = list(sindex.intersection(row[geometry_col_name].bounds))
        possible_matches = current_gdf.iloc[possible_matches_index]

        for other_idx, other in possible_matches.iterrows():
            if idx != other_idx and row[geometry_col_name].within(other[geometry_col_name]):
                return False
            elif idx != other_idx and current_centroid.within(other[geometry_col_name]) and row['Hylak_id'] > other['Hylak_id']:
                return False

        return True

    # Check containment in parallel
    # Apply the containment check in parallel
    keep_mask = current_gdf.parallel_apply(lambda row: check_containment(row.name, row), axis=1)

    # Filter the GeoDataFrame to keep only non-contained geometries
    gdf_filtered = current_gdf[keep_mask].reset_index(drop=True)


    # Reset index of the resulting GeoDataFrame
    gdf_filtered.reset_index(drop=True, inplace=True)
    
    gdf_filtered.to_file(os.path.join(hylak_no_contained_save_folder, hylak_no_contained_save_filename_pattern.format(hybas_id=hybas_id)))


Initializing pandarallel...
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Processing 1020000010
Processing 1020011530
Processing 1020018110
Processing 1020021940
Processing 1020027430
Processing 1020034170
Processing 1020035180
Processing 1020040190
Processing 2020000010
Processing 2020003440
Processing 2020018240
Processing 2020024230
Processing 2020033490
Processing 2020041390
Processing 2020057170
Processing 2020065840
Processing 2020071190
Processing 3020000010
Processing 3020003790
Processing 3020005240
Processing 3020008670
Processing 3020009320
Processing 3020024310
Processing 4020000010
Processing 4020006940
Processing 4020015090
Processing 4020024190
Processing 4020034510
Processing 4020050210
Processing 4020050220
Processing 4020050290
Processing 4020050470
Processing 5020000010
Processing 5020015660
Processing 5020037270
Processing 5020049720
Processing

In [2]:
def read_wkt(file_path):
    # Step 1: Read the WKT string from the file
    with open(file_path, 'r') as file:
        wkt_string = file.read()
    return wkt_string

def calculate_buffer_size(row, lake_area_min, lake_area_max, slope_100_min, slope_100_max, buffer_min, buffer_max):
    slope_100 = row['Slope_100']
    lake_area = row['Lake_area']
    buffer_adjust_ratio = (lake_area - lake_area_min) / (lake_area_max - lake_area_min)
    if buffer_adjust_ratio < 0:
        buffer_adjust_ratio = 0
    if buffer_adjust_ratio > 1:
        buffer_adjust_ratio = 1
    slope_inflation_factor = 0.5 if slope_100 == -1 else (slope_100_max - slope_100) / (slope_100_max - slope_100_min)
    slope_inflation_factor += 1
    buffer_size = (buffer_min + buffer_adjust_ratio * (buffer_max - buffer_min)) * slope_inflation_factor
    return buffer_size

from pandarallel import pandarallel
print('Initializing pandarallel...')
pandarallel.initialize(progress_bar=False, nb_workers=12, use_memory_fs = False)

hybas_id_list = [
    1020000010, 1020011530, 1020018110, 1020021940, 1020027430, 1020034170, 1020035180, 1020040190,
    2020000010, 2020003440, 2020018240, 2020024230, 2020033490, 2020041390, 2020057170, 2020065840, 2020071190,
    3020000010, 3020003790, 3020005240, 3020008670, 3020009320, 3020024310,
    4020000010, 4020006940, 4020015090, 4020024190, 4020034510, 4020050210, 4020050220, 4020050290, 4020050470,
    5020000010, 5020015660, 5020037270, 5020049720, 5020082270, 
    6020000010, 6020006540, 6020008320, 6020014330, 6020017370, 6020021870, 6020029280,
    7020000010, 7020014250, 7020021430, 7020024600, 7020038340, 7020046750, 7020047840, 7020065090,
    8020000010, 8020008900, 8020010700, 8020020760, 8020022890, 8020032840, 8020044560,
    9020000010
]
hylak_folder = '/WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new'
hylak_filename_pattern = 'hylak_unbuffered_updated_no_contained_{hybas_id}.shp'
projection_folder = '/WORK/Codes/global_lake_area/projection_wkt/Lambert_Azimuthal_Equal_Area'
projection_filename_pattern = '{hybas_id}.wkt'
hylak_buffered_save_folder = '/WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_buffered'
if not os.path.exists(hylak_buffered_save_folder):
    os.makedirs(hylak_buffered_save_folder)
hylak_buffered_filename_pattern = 'hylak_buffered_updated_no_contained_{hybas_id}.shp'

use_parallel = True

for hybas_id in hybas_id_list:
    current_hylak_path = os.path.join(hylak_folder, hylak_filename_pattern.format(hybas_id=hybas_id))
    print(f'Processing {current_hylak_path}...')
    current_hylak_gdf = gpd.read_file(current_hylak_path)
    lake_area_min = 0.1
    lake_area_max = 1000
        
    slope_100_min = current_hylak_gdf['Slope_100'].min()
    slope_100_max = current_hylak_gdf['Slope_100'].max()
    buffer_min = 90
    buffer_max = 2250
    print(f'Caculating buffer size for {current_hylak_path}...')
    if use_parallel:
        current_hylak_gdf['Buffer_size'] = current_hylak_gdf.parallel_apply(lambda row: calculate_buffer_size(row, lake_area_min, lake_area_max, slope_100_min, slope_100_max, buffer_min, buffer_max), axis=1)
    else:
        current_hylak_gdf['Buffer_size'] = current_hylak_gdf.apply(lambda row: calculate_buffer_size(row, lake_area_min, lake_area_max, slope_100_min, slope_100_max, buffer_min, buffer_max), axis=1)
    
    current_projection_wkt_path = os.path.join(projection_folder, projection_filename_pattern.format(hybas_id=hybas_id))
    current_projection_wkt = read_wkt(current_projection_wkt_path)
    current_projection = CRS(current_projection_wkt)
    
    hylak_gdf_original_crs = current_hylak_gdf.crs
    current_hylak_gdf = current_hylak_gdf.to_crs(current_projection)
    
    print(f'Simplifying {current_hylak_path}...')
    if use_parallel:
        current_hylak_gdf['geometry'] = current_hylak_gdf.parallel_apply(lambda row: row['geometry'].simplify(120), axis=1) # 60 meters
    else:
        current_hylak_gdf['geometry'] = current_hylak_gdf.simplify(120) # 60 meters
    print(f'Buffering {current_hylak_path}...')
    if use_parallel:
        current_hylak_gdf['geometry'] = current_hylak_gdf.parallel_apply(lambda row: row['geometry'].buffer(row['Buffer_size']), axis=1)
    else:
        current_hylak_gdf['geometry'] = current_hylak_gdf.buffer(current_hylak_gdf['Buffer_size'])
    current_hylak_gdf = current_hylak_gdf.to_crs(hylak_gdf_original_crs)
    
    current_hylak_buffered_save_path = os.path.join(hylak_buffered_save_folder, hylak_buffered_filename_pattern.format(hybas_id=hybas_id))
    current_hylak_gdf.to_file(current_hylak_buffered_save_path)
    

Initializing pandarallel...
INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Processing /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_1020000010.shp...
Caculating buffer size for /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_1020000010.shp...
Simplifying /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_1020000010.shp...
Buffering /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_1020000010.shp...


/tmp/ipykernel_61995/1773437933.py:83: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  current_hylak_gdf.to_file(current_hylak_buffered_save_path)


Processing /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_1020011530.shp...
Caculating buffer size for /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_1020011530.shp...
Simplifying /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_1020011530.shp...
Buffering /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_1020011530.shp...


/tmp/ipykernel_61995/1773437933.py:83: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  current_hylak_gdf.to_file(current_hylak_buffered_save_path)


Processing /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_1020018110.shp...
Caculating buffer size for /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_1020018110.shp...
Simplifying /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_1020018110.shp...
Buffering /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_1020018110.shp...


...

Processing /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_8020044560.shp...
Caculating buffer size for /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_8020044560.shp...
Simplifying /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_8020044560.shp...
Buffering /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_8020044560.shp...


/tmp/ipykernel_61995/1773437933.py:83: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  current_hylak_gdf.to_file(current_hylak_buffered_save_path)


Processing /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_9020000010.shp...
Caculating buffer size for /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_9020000010.shp...
Simplifying /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_9020000010.shp...
Buffering /WORK/Data/global_lake_area/lake_shps/HydroLAKES_updated_using_GLAKES/per_basin_no_contained_new/hylak_unbuffered_updated_no_contained_9020000010.shp...


/tmp/ipykernel_61995/1773437933.py:83: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  current_hylak_gdf.to_file(current_hylak_buffered_save_path)


: 